In [42]:

from openai import OpenAI
import json
from pykrx import stock
from pykrx import bond

client = OpenAI()

In [43]:
#종목별 DIV/BPS/PER/EPS 조회
def get_market_fundamental(start_dt, end_dt, ticket):
    df = stock.get_market_fundamental(start_dt, end_dt, ticket)
    return json.dumps(df.head(365).to_dict(orient="records"), ensure_ascii=False)

#일자별 OHLCV 조회
def get_market_ohlcv(start_dt, end_dt, ticket):
    df = stock.get_market_ohlcv(start_dt, end_dt, ticket)
    return json.dumps(df.head(365).to_dict(orient="records"), ensure_ascii=False)

In [44]:
tickers = stock.get_market_ticker_list()
종목 = {}

for ticker in tickers:
    종목[stock.get_market_ticker_name(ticker)] = ticker

def get_stock_info(start_dt, end_dt, ticket):
    data = {}
    data["fundamental"] = get_market_fundamental(start_dt, end_dt, ticket)
    data["ohlcv"] = get_market_ohlcv(start_dt, end_dt, ticket)

    return data

messages = [{"role": "user", "content": "2024년 1월 LG의 주식 정보를 요약해줘"}]
functions = [
    {
        "name": "get_stock_info",
        "description": "특정 지역의 날씨를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "start_dt": {
                    "type": "string",
                    "description": "시작일자, ex) 2024년이라고 말한 경우 2024년 1월 1일, 2024년 5월이라고 말한 경우 2024년 5월 1일",
                },
                "end_dt": {
                    "type": "string",
                    "description": "종료일자, ex) 2024년이라고 말한 경우 2024년 1월 1일, 2024년 5월이라고 말한 경우 2024년 5월 1일",
                },
                "ticker": {
                    "type": "string", 
                    "description": str(종목)
                },
            },
            "required": ["start_dt", "end_dt", "ticker"],
        },
    }
]

In [45]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="auto",
    )
response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='{"start_dt":"2024-01-01","end_dt":"2024-01-02","ticker":"066570"}', name='get_stock_info'), tool_calls=None)


In [46]:
if response_message.function_call is not None:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_stock_info": get_stock_info,
    }
    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        start_dt=function_args['start_dt'],
        end_dt=function_args['end_dt'],
        ticker=function_args['ticker'],
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )  # get a new response from GPT where it can see the function response

    print(second_response.choices[0].message.content)

NameError: name 'get_current_weather' is not defined